---

### Promemoria: questo 📘 `.NET Interactive` deve essere eseguito da VS Code con [questi prerequisiti](../PREREQS.md).

---

<!-- #### How to use this notebook: 

* Just read the text and scroll along until you run into code blocks.
* Code blocks have computer code inside them — hover over the block and you can run the code.
* Run the code by hitting the ▶️ "play" button to the left. If the code runs you'll see a ✔️. If not, you'll get a ❌.
* The output and status of the code block will appear just below itself — you need to scroll down further to see it.
* Sometimes a code block will ask you for input in a hard-to-notice dialog box 👆 at the top of your notebook window.  -->

<!-- --- -->

# Ricetta V: 🧄 Connettori con Bing
## 🧑‍🍳 Cucinare utilizzando un contesto con dati in tempo reale dalle API

## Step 1: Instanziare un 🔥 kernel 

In [40]:
#r "nuget: Microsoft.SemanticKernel, 0.9.61.1-preview"
#r "nuget: Microsoft.SemanticKernel.Skills.Web, 0.9.61.1-preview"

#!import ../config/Settings.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.KernelExtensions;
using Microsoft.SemanticKernel.Configuration;
using Microsoft.SemanticKernel.SemanticFunctions;
using Microsoft.SemanticKernel.Orchestration;

IKernel kernel = Microsoft.SemanticKernel.Kernel.Builder.Build();

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    kernel.Config.AddAzureOpenAITextCompletion("davinci", model, azureEndpoint, apiKey);
else
    kernel.Config.AddOpenAITextCompletion("davinci", model, apiKey, orgId);

Installed Packages Microsoft.SemanticKernel, 0.9.61.1-preview Microsoft.SemanticKernel.Skills.Web, 0.9.61.1-preview

## Step 2: Integrare i dati in tempo reale con l'API di ricerca web di Bing 🧄 utilizzando il Connector

Per ottenere una chiave API di Bing Web Search, visitate [questa risorsa](https://www.microsoft.com/en-us/bing/apis/bing-web-search-api) per iniziare.\
A questo punto si possono prendere le informazioni della ricerca in tempo reale e aggiungerle al contesto scelto.\
Ad esempio, possiamo scoprire qual è l'edificio più alto d'Europa:

In [70]:
using Microsoft.SemanticKernel.Skills.Web;
using Microsoft.SemanticKernel.Skills.Web.Bing;
using Newtonsoft.Json;

// var bingApiKey = await InteractiveKernel.GetPasswordAsync("Please enter your Bing Search API key");
var bingApiKey = File.ReadAllText("../config/settings.json");
dynamic settings = JsonConvert.DeserializeObject(File.ReadAllText("../config/settings.json"));
bingApiKey =  settings["bingApiKey"];

var bingConnector = new BingConnector(bingApiKey);

var webSearchEngineSkill = new WebSearchEngineSkill(bingConnector);
var web = kernel.ImportSkill(webSearchEngineSkill);

var bingResult = await kernel.RunAsync(
    "What's the tallest building in NYC?", 
    web["SearchAsync"]
);
Console.WriteLine(bingResult);

62469983026a49dd91a88de80e898009


> ✅ Tenete presente che per ottenere una chiave API di Bing Web Search è necessario disporre di un account Azure gratuito o di un account Azure a pagamento con credenziali di fatturazione attive.

> 🤔 **Ottieni il seguente `"Error: Response status code does not indicate success: 401 (PermissionDenied)."` messaggio?** Tpotrebbe accadere se si è appena effettuato il provisioning di un'API di Bing Web Search. Attendere qualche minuto e riprovare.

<!-- > 😱 **Get a different error message?** go to https://aka.ms/sk/discord where we have realtime support available to troubleshoot your problem. -->

## Step 3: Combiniamo 🍝 memories esistenti with nuovi 🛢️ dati

Portiamo le nostre 🍝 **Memories** a un livello superiore, fornendogli memories "live" che vengono forniti tramite un 🧄 **Connector**.\
Inseriamo l'output della funzione `SearchAsync` in una ⚡ **Funzione Semantica** inline creata su misura per raccontare una breve storia, includendo alcuni "fatti" come 🍝 Memories inline , come:

1. I'm tall-ish: about 5ft 11in | `"Sono piuttosto alto: circa 1 metro e 80."`
2. I like sushi — but not the kind with uni | `"Mi piace il sushi - ma non quello con l'uni"`

E aggiungiamo un ricordo fresco che prendiamo da un 🧄 Bing connettor per vantarci di essere stati sull'`edificio più alto di New York`, aggiungendo un terzo dato:

3. I've been to `<the tallest building in NYC>`

In [5]:
var myInlineSemanticFunction = kernel.CreateSemanticFunction(@"
These are facts about me:
1. I'm tall-ish: about 5ft 11in
2. I like sushi — but not the kind with uni
3. I've been to {{$input}}
Here is a short story under 100 words about me:
", maxTokens: 200, temperature: 0.1, topP: .1);

var result = await kernel.RunAsync(
    "What's the tallest building in NYC?", 
    web["SearchAsync"],
    myInlineSemanticFunction
);

Console.WriteLine(result);


I'm a tall-ish person, standing at 5ft 11in. I love sushi, but not the kind with uni. I've been to the tallest building in the United States, One World Trade Center, which stands at 1,776 feet. It's a 104-story skyscraper and the tallest building in the Western Hemisphere. I'm always looking for new experiences and I'm excited to see what the future holds.


Ora potete vedere come è possibile utilizzare Semantic Kernel in vari modi: dalla progettazione di prompt, all'espansione del contesto di un prompt con le memories, fino all'inserimento di dati in tempo reale.

## Step 4: Facciamo un ulteriore passo avanti e aggiungiamo un po' di condimento con la `SummarizeSkill.Notegen` ⚡ function

In [6]:
var skillsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "skills");
var mySkill = kernel.ImportSemanticSkillFromDirectory(skillsDirectory, "SummarizeSkill");

var result = await kernel.RunAsync(
    "What's the tallest building in NYC?", 
    web["SearchAsync"],
    myInlineSemanticFunction,
    mySkill["Notegen"]
);

// Return the result to the Notebook
Console.WriteLine(result);

Personal Info
- 5ft 11in
- Likes sushi, not uni
- Visited One World Trade Center (1,776ft, 104-story)
- Always looking for new experiences


Se si osserva attentamente il codice, abbiamo creato una "catena" di funzioni.

<!-- # ⏭️ Next Steps

Run through more advanced examples in the notebooks that are available in our GitHub repo at [https://aka.ms/sk/repo](https://aka.ms/sk/repo).

[Design thinking with 🔥 kernel is up next!](../e6-design-chain/notebook.ipynb)

Or stay a longer while and ask Bing to added new information from the Internet to fortify your LLM AI magic. And you can also try different functions within the `SummarizeSkill` to see different outputs in Step 4. -->